In [101]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertConfig, BertModel, BertPreTrainedModel
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [102]:
# Sample Data
data = {
    'prompt': [
        "Create a chatbot script for customer support",
        "Generate a response for mental health chat",
        "Write a conversation starter for virtual assistants",
        "Generate an image of a sunset over the ocean",
        "Create a video clip of a relaxing forest scene",
        "Develop a conversational flow for a virtual assistant handling scheduling",
        "Generate a supportive message for someone experiencing anxiety",
        "Write an introduction script for a virtual tour guide",
        "Create a realistic image of a mountain landscape",
        "Produce a short video of a cityscape at night",
        "Compose a background score for a meditation session",
        "Generate a speech script for a company announcement",
        "Create podcast episode outlines about technology trends",
        "Write a blog post on the benefits of remote work",
        "Generate a podcast script discussing climate change",
        "Develop a story outline for a children's adventure book",
        "Create social media posts for a new product launch",
        "Write product descriptions for an online store",
        "Optimize a website's content for search engines",
        "Generate a market analysis report for a new app",
        "Translate a document from English to Spanish",
        "Provide coding assistance for a Python project",
        "Integrate a third-party API into a web application",
        "Analyze sales data and generate a report",
        "Create a financial report based on quarterly earnings",
        "Develop educational content for an online course",
        "Assist in learning Spanish vocabulary",
        "Create a PowerPoint presentation on digital marketing",
        "Generate email templates for business communication",
        "Draft a legal document for a lease agreement",
        "Generate a contract for freelance work",
        "Create a healthy meal plan for a week",
        "Generate a fitness routine for beginners",
        "Solve math homework problems",
        "Detect AI-generated content in student essays",
        "Design a logo for a startup company",
        "Create a branding strategy for a new brand",
        "Plan a travel itinerary for a trip to Europe",
        "Organize an event plan for a wedding",
        "Write detailed product descriptions for an online shop",
        "Generate personalized product recommendations",
        "Draft a resume for a job application",
        "Write a cover letter for a software developer position",
        "Summarize data for a research paper",
        "Generate a summary of the latest news articles",
        "Create a news article about a recent scientific discovery",
        "Generate a route for an LLM handling diverse queries"
    ],
    'category': [
        "Communication",
        "Communication",
        "Communication",
        "Visual Art",
        "Visual Art",
        "Communication",
        "Communication",
        "Communication",
        "Visual Art",
        "Visual Art",
        "Music and Audio",
        "Music and Audio",
        "Music and Audio",
        "Writing and Content Creation",
        "Writing and Content Creation",
        "Writing and Content Creation",
        "Marketing and Advertising",
        "Marketing and Advertising",
        "Marketing and Advertising",
        "Marketing and Advertising",
        "Translation and Localization",
        "Programming and Development",
        "Programming and Development",
        "Data and Analytics",
        "Data and Analytics",
        "Education and Training",
        "Education and Training",
        "Business and Productivity",
        "Business and Productivity",
        "Legal and Professional Services",
        "Legal and Professional Services",
        "Health and Wellness",
        "Health and Wellness",
        "Homework",
        "Homework",
        "Design",
        "Design",
        "Travel and Hospitality",
        "Travel and Hospitality",
        "Retail and E-commerce",
        "Retail and E-commerce",
        "Human Resources",
        "Human Resources",
        "Science and Research",
        "Science and Research",
        "Media and Journalism",
        "Others"
    ],
    'subcategory': [
        "Chatbots and Virtual Assistants",
        "Mental health",
        "Conversation",
        "Image Generation",
        "Video Generation",
        "Chatbots and Virtual Assistants",
        "Mental health",
        "Conversation",
        "Image Generation",
        "Video Generation",
        "Music Creation",
        "Speech Generation",
        "Podcast Content Creation",
        "Blog Writing",
        "Podcasts",
        "Storytelling and Narrative Creation",
        "Social Media Posts",
        "Product Descriptions",
        "SEO Optimization",
        "Market Analysis",
        "Translation",
        "Coding and Programming Assistance",
        "API Integration",
        "Data Analysis",
        "Financial Report Generation",
        "Educational Content Creation",
        "Language Learning Assistance",
        "Presentation Creation",
        "Email Generation",
        "Legal Document Drafting",
        "Contract Generation",
        "Recipe Generation",
        "Fitness Routine Generation",
        "Homework Assistance",
        "AI Content Detection",
        "Logo Design",
        "Branding Strategy",
        "Travel Itinerary Planning",
        "Event Planning",
        "Product Description Writing",
        "Personalized Recommendations",
        "Resume Drafting",
        "Cover Letter Writing",
        "Data Summarization",
        "News Summary Generation",
        "News Article Creation",
        "LLM Route Generation"
    ]
}

In [103]:
# Convert to DataFrame
df = pd.DataFrame(data)

In [104]:
texts = df['prompt'].values
categories = df['category'].values
subcategories = df['subcategory'].values

In [105]:
# Tokenization for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
X_bert = tokenizer(texts.tolist(), padding=True, truncation=True, max_length=50, return_tensors='pt')
X_bert_ids = X_bert['input_ids']
X_bert_masks = X_bert['attention_mask']

In [106]:
# Encode labels
label_encoder_cat = LabelEncoder()
y_cat = label_encoder_cat.fit_transform(categories)

label_encoder_subcat = LabelEncoder()
y_subcat = label_encoder_subcat.fit_transform(subcategories)

In [107]:
# Split data
X_train_ids, X_test_ids, y_train_cat, y_test_cat, y_train_subcat, y_test_subcat = train_test_split(
    X_bert_ids.numpy(), y_cat, y_subcat, test_size=0.2, random_state=42
)
X_train_masks, X_test_masks, _, _ = train_test_split(
    X_bert_masks.numpy(), X_bert_masks.numpy(), test_size=0.2, random_state=42
)

In [108]:
# Define Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_masks, category_labels, subcategory_labels):
        self.input_ids = input_ids
        self.attention_masks = attention_masks
        self.category_labels = category_labels
        self.subcategory_labels = subcategory_labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
            'category_label': torch.tensor(self.category_labels[idx], dtype=torch.long),
            'subcategory_label': torch.tensor(self.subcategory_labels[idx], dtype=torch.long)
        }
        return item

In [109]:
# Create dataset instances
train_dataset = CustomDataset(
    input_ids=X_train_ids.tolist(),
    attention_masks=X_train_masks.tolist(),
    category_labels=y_train_cat.tolist(),
    subcategory_labels=y_train_subcat.tolist()
)

In [111]:
# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [112]:
# Define LSTM model
def create_lstm_model(input_length, vocab_size, embedding_dim, num_labels):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
        LSTM(128),
        Dense(num_labels, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [113]:
# Prepare LSTM data
max_length = 50
X_bert_padded = pad_sequences(X_bert_ids.numpy(), maxlen=max_length)
vocab_size = tokenizer.vocab_size
embedding_dim = 50

In [130]:
# Train LSTM model
lstm_model = create_lstm_model(max_length, vocab_size, embedding_dim, len(label_encoder_cat.classes_))
history_lstm = lstm_model.fit(
    X_bert_padded, y_cat,
    epochs=100,
    batch_size=8,
    validation_split=0.1
)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 190ms/step - accuracy: 0.0252 - loss: 2.9966 - val_accuracy: 0.0000e+00 - val_loss: 3.0284
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.1292 - loss: 2.9727 - val_accuracy: 0.0000e+00 - val_loss: 3.0850
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.2081 - loss: 2.9399 - val_accuracy: 0.0000e+00 - val_loss: 3.2839
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.1325 - loss: 2.8803 - val_accuracy: 0.0000e+00 - val_loss: 4.4321
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.1783 - loss: 2.8146 - val_accuracy: 0.0000e+00 - val_loss: 4.9220
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.1024 - loss: 2.8225 - val_accuracy: 0.0000e+00 - val_loss: 4.2863
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.1337 - loss: 2.7397 - val_accuracy: 0.0000e+00 - val_loss: 4.3027
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.1381 - loss: 2.7630 - val_accur

In [115]:
# Define GRU model
def create_gru_model(input_length, vocab_size, embedding_dim, num_labels):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_length),
        GRU(128),
        Dense(num_labels, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [131]:
# Train GRU model
gru_model = create_gru_model(max_length, vocab_size, embedding_dim, len(label_encoder_cat.classes_))
history_gru = gru_model.fit(
    X_bert_padded, y_cat,
    epochs=100,
    batch_size=8,
    validation_split=0.1
)

Epoch 1/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 178ms/step - accuracy: 0.0832 - loss: 2.9959 - val_accuracy: 0.0000e+00 - val_loss: 3.0301
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.1099 - loss: 2.9722 - val_accuracy: 0.0000e+00 - val_loss: 3.0765
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1619 - loss: 2.9438 - val_accuracy: 0.0000e+00 - val_loss: 3.1381
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.1500 - loss: 2.9222 - val_accuracy: 0.0000e+00 - val_loss: 3.2301
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - accuracy: 0.1233 - loss: 2.8926 - val_accuracy: 0.0000e+00 - val_loss: 3.3798
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.1203 - loss: 2.8657 - val_accuracy: 0.0000e+00 - val_loss: 3.6905
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.2081 - loss: 2.7457 - val_accuracy: 0.0000e+00 - val_loss: 4.1091
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.1426 - loss: 2.8094 - val

In [133]:
# Define BERT model
class BertForTextClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(config.hidden_size, len(label_encoder_cat.classes_))
        self.subclassifier = nn.Linear(config.hidden_size, len(label_encoder_subcat.classes_))

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        sub_logits = self.subclassifier(pooled_output)
        return logits, sub_logits

In [134]:
# Initialize and train BERT model
config = BertConfig.from_pretrained('bert-base-uncased')
bert_model = BertForTextClassification(config)

In [135]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(bert_model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

In [136]:
# Training loop for BERT
bert_model.train()
for epoch in range(50):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        category_labels = batch['category_label']
        subcategory_labels = batch['subcategory_label']

        logits, sub_logits = bert_model(input_ids, attention_mask=attention_mask)
        loss_cat = loss_fn(logits, category_labels)
        loss_subcat = loss_fn(sub_logits, subcategory_labels)
        loss = loss_cat + loss_subcat
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

Epoch 1: Loss = 6.735448837280273
Epoch 2: Loss = 6.722023963928223
Epoch 3: Loss = 6.733700752258301
Epoch 4: Loss = 6.648200035095215
Epoch 5: Loss = 6.283683776855469
Epoch 6: Loss = 5.719743728637695
Epoch 7: Loss = 6.4594268798828125
Epoch 8: Loss = 5.639106750488281
Epoch 9: Loss = 6.060833930969238
Epoch 10: Loss = 5.759893417358398
Epoch 11: Loss = 5.970782279968262
Epoch 12: Loss = 5.234567642211914
Epoch 13: Loss = 5.895871639251709
Epoch 14: Loss = 5.5872697830200195
Epoch 15: Loss = 5.972634315490723
Epoch 16: Loss = 5.353873252868652
Epoch 17: Loss = 5.580223083496094
Epoch 18: Loss = 5.456847190856934
Epoch 19: Loss = 4.879000663757324
Epoch 20: Loss = 4.1474409103393555
Epoch 21: Loss = 3.846884250640869
Epoch 22: Loss = 4.843586444854736
Epoch 23: Loss = 3.8165626525878906
Epoch 24: Loss = 3.9803788661956787
Epoch 25: Loss = 4.0972514152526855
Epoch 26: Loss = 2.983478546142578
Epoch 27: Loss = 3.6032981872558594
Epoch 28: Loss = 3.2951605319976807
Epoch 29: Loss = 3.13

In [137]:
# Function to predict using all models separately
def predict_separate_models(texts):
    # Prepare input for BERT
    bert_inputs = tokenizer(texts, padding=True, truncation=True, max_length=50, return_tensors='pt')
    input_ids = bert_inputs['input_ids'].clone().detach()
    attention_mask = bert_inputs['attention_mask'].clone().detach()

    # Predict with LSTM
    X_bert_padded_test = pad_sequences(tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')['input_ids'].numpy(), maxlen=max_length)
    lstm_preds = lstm_model.predict(X_bert_padded_test)

    # Predict with GRU
    gru_preds = gru_model.predict(X_bert_padded_test)

    # Predict with BERT
    with torch.no_grad():
        logits, sub_logits = bert_model(input_ids, attention_mask=attention_mask)
        bert_preds_cat = torch.argmax(logits, dim=1).numpy()
        bert_preds_subcat = torch.argmax(sub_logits, dim=1).numpy()

    # Return predictions separately
    return lstm_preds, gru_preds, bert_preds_cat, bert_preds_subcat

In [140]:
# Predict using all models
texts_to_predict = ["Plan a travel itinerary for a trip to Europe"]
lstm_preds, gru_preds, bert_preds_cat, bert_preds_subcat = predict_separate_models(texts_to_predict)

# Convert predictions to category and subcategory labels
lstm_categories = np.argmax(lstm_preds, axis=1)
gru_categories = np.argmax(gru_preds, axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [141]:
print(f"LSTM Predicted Categories: {label_encoder_cat.inverse_transform(lstm_categories)}")
print(f"LSTM Predicted Subcategories: {label_encoder_subcat.inverse_transform(np.argmax(lstm_preds, axis=1))}")

print(f"GRU Predicted Categories: {label_encoder_cat.inverse_transform(np.argmax(gru_preds, axis=1))}")
print(f"GRU Predicted Subcategories: {label_encoder_subcat.inverse_transform(np.argmax(gru_preds, axis=1))}")

print(f"BERT Predicted Categories: {label_encoder_cat.inverse_transform(bert_preds_cat)}")
print(f"BERT Predicted Subcategories: {label_encoder_subcat.inverse_transform(bert_preds_subcat)}")

LSTM Predicted Categories: ['Travel and Hospitality']
LSTM Predicted Subcategories: ['Image Generation']
GRU Predicted Categories: ['Travel and Hospitality']
GRU Predicted Subcategories: ['Image Generation']
BERT Predicted Categories: ['Travel and Hospitality']
BERT Predicted Subcategories: ['Travel Itinerary Planning']
